In [7]:
import pandas as pd
import traceback
from fuzzywuzzy import fuzz, process
from collections import defaultdict
import gensim
import datetime
from datetime import datetime as dt, timedelta as td
import re
import textstat
from textstat.textstat import textstat
import nltk
from itertools import chain
import pandasql as ps
# from BaseAD import CreateBaseAD,QuarterIndex ,get_quarteryr
# from qaextraction import get_analysts_from_preamble
import pandas as pd
import itertools
import time
from utils import get_date_from_preamble
import gc
import numpy as np
%load_ext autoreload
%autoreload 2

In [8]:
## Read Data From Output of join script 

set1 = pd.read_csv('D:\\Chubb\\Data\\Output\\transcript_with_cals_1901_V1_1.csv',sep='|')
set2 = pd.read_csv('D:\\Chubb\\Data\\Output\\transcript_with_cals_1901_V1_2.csv',sep='|')
set3 = pd.read_csv('D:\\Chubb\\Data\\Output\\transcript_with_cals_1901_V1_3.csv',sep='|')
set4 = pd.read_csv('D:\\Chubb\\Data\\Output\\transcript_with_cals_1901_V1_4.csv',sep='|')                  
# set2 = pd.read_csv('data/transcript_with_cals_1901_V1_2.csv',sep='|')
# set3 = pd.read_csv('data/transcript_with_cals_1901_V1_3.csv',sep='|')
# set4 = pd.read_csv('data/transcript_with_cals_1901_V1_4.csv',sep='|')


data_raw = set1.append([set2, set3, set4])

qa_len_list= [len(set1), len(set2), len(set3), len(set4)]

In [9]:
## Dropping intermediate files and clearing cache

del set1
del set2
del set3
del set4

gc.collect()

5402

In [10]:
np.sum(qa_len_list)

48844

In [11]:
data_raw["date_earning_call"] = data_raw["date_earning_call"].apply(pd.to_datetime)
data_raw["date_earning_call"] = data_raw["date_earning_call"].apply(pd.Timestamp.date)

In [12]:
data_raw = data_raw[['input_company_name2', 'About',
       'Transcript_page_heading', 'Transcript_heading', 'Preamble',
       'transcript_after_preamble', 'date_earning_call', 
       'year', 'published_date_time', 'next_publish_date', 'def_date',
       'day_diff', 'cal_def_value', 'filing_date_2', 'is_cals']]


data_raw = data_raw.drop_duplicates()

data_raw = data_raw[~data_raw['Preamble'].isnull()]
len(data_raw)

48844

In [13]:
#EDA KIND
# Create key at date and company level

#data_raw['date_earning_call_str'] = [str(i) for i in list(data_raw['date_earning_call'])]

data_raw['date_earning_call_str'] = data_raw['date_earning_call'].apply(str)
data_raw['key'] = data_raw['input_company_name2'] + data_raw['date_earning_call_str']
data_raw['Count'] = data_raw.groupby('key')['key'].transform('count')
#data_raw['max_published_date'] = data_raw.groupby('key')['published_date_time'].transform('max')

In [14]:
#Remove duplicates at earning date and compnay level

## Step 1 - Crete a subset of data whicch are having multiple counts at Key level 
data_temp = data_raw[data_raw['Count']>1 ]
data_temp['index'] = data_temp.index
data_temp['rank'] = data_temp.groupby('key')['index'].rank(ascending=1)
data_temp2 = data_temp[data_temp['rank'] == 1]
data_temp2 = data_temp2.drop(['rank','index'],axis = 1)
data_temp1 = data_raw[data_raw['Count']==1]
data = data_temp1.append(data_temp2,ignore_index =True)

C:\Users\abhishek\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\abhishek\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
## Deleting intermediaries

del data_temp
del data_temp1
del data_temp2
del data_raw

gc.collect()

7

In [16]:
data["next_earning_date"] =  data.sort_values(by='date_earning_call').groupby(['input_company_name2'])['date_earning_call'].shift(-1)

# 1 quarter previous transcipts 
data["1Q_Back"] =  data.sort_values(by='date_earning_call').groupby(['input_company_name2'])['date_earning_call'].shift(1)

#2 quarter previous transcipts 
data["2Q_Back"] =  data.sort_values(by='date_earning_call').groupby(['input_company_name2'])['date_earning_call'].shift(2)

# 3 quarter previous transcipts 
data["3Q_Back"] =  data.sort_values(by='date_earning_call').groupby(['input_company_name2'])['date_earning_call'].shift(3)

# 4 quarter previous transcipts 
data["4Q_Back"] =  data.sort_values(by='date_earning_call').groupby(['input_company_name2'])['date_earning_call'].shift(4)

# 5 quarter previous transcipts 
data["5Q_Back"] =  data.sort_values(by='date_earning_call').groupby(['input_company_name2'])['date_earning_call'].shift(5)

# 6 quarter previous transcipts 
data["6Q_Back"] =  data.sort_values(by='date_earning_call').groupby(['input_company_name2'])['date_earning_call'].shift(6)

# 7 quarter previous transcipts 
data["7Q_Back"] =  data.sort_values(by='date_earning_call').groupby(['input_company_name2'])['date_earning_call'].shift(7)


 

In [17]:
df = data
del(data)

gc.collect()

49

In [18]:
## Runninng some preprocessing steps

df = df.rename(columns = {'input_company_name2':'Company'})
df = df.rename(columns = {'transcript_after_preamble':'QA'})
df['combined']=df['Preamble'].astype(str)+df['QA'].astype(str)
df['Size'] = [len(x) for x in df.loc[:,'combined']] 
df['Size_Preamble'] = [len(x) for x in df.loc[:,'Preamble']]
df['Size_QA'] = [len(x) for x in df.loc[:,'QA']]

In [31]:
## Import textutils features

from generatefeatures import TextUtils,GenerateFeatures
from utils import TextUtils
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
textutils = TextUtils(stopword_file='D:\\Chubb\\Data\\Input\\final_stopword list.csv', use_spacy=True)
generatefeatures = GenerateFeatures(textutils=textutils, lm_filepath='D:\\Chubb\\Data\\Input\\LoughranMcDonald_MasterDictionary_2016.csv')

C:\Users\abhishek\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: DtypeWarning: Columns (63,108,109,110,176) have mixed types. Specify dtype option on import or set low_memory=False.
  


In [ ]:
st = time.time()
df['QA_features']= df['QA'].apply(generatefeatures.generate_features)
df['Preamble_features'] = df['Preamble'].apply(generatefeatures.generate_features)
df['Combined_features'] = df['combined'].apply(generatefeatures.generate_features)
ed = time.time()
tt = ed-st

C:\Users\abhishek\Anaconda3\envs\tensorflow\lib\site-packages\numpy\lib\function_base.py:1128: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
